# Playground

## Import Libraries

In [1]:
import pandas as pd
import plotly.graph_objs as go
import plotly.offline as offline
import os
import numpy as np
from plotly.offline import plot
from pathlib import Path

In [2]:
offline.init_notebook_mode(connected=True)

# Dot Plot of Net Zero Targets by Sector

In [3]:
path = Path(os.path.dirname (os.getcwd()))
XLSX_PATH = os.path.join(path.parent.parent.parent, 'sp100.xlsx')

## Compute total companies count and revenue by sector

In [4]:
# total number of companies
total_companies_count = 100

# total revenue of all companies

COLS_TO_USE = {
'companies':['company_id', 'SP100'],
'financials':['company_id', 'Revenue_num_tradingview'],
'grouping':['company_id', 'Sector1'],
}

all_df = []
for sheetname, cols in COLS_TO_USE.items():
    all_df.append(pd.read_excel(
        XLSX_PATH, 
        sheet_name = sheetname,
        engine = 'openpyxl', 
        usecols = cols
        ))
merged_df = all_df[0]
for i in range(len(COLS_TO_USE) - 1):
    merged_df = pd.merge(
        left = merged_df,
        right = all_df[i+1],
        how="left",
        on="company_id"
    )
# Select Final Figures and Year 2019

cond1 = (merged_df['SP100'] == 1)

filter_cond = cond1 #& cond2
filter_cond = filter_cond
merged_df = merged_df.loc[filter_cond][['Sector1', 'company_id', 'Revenue_num_tradingview']]

total_companies_revenue_by_sector = merged_df[['Sector1', 'Revenue_num_tradingview']].groupby('Sector1').sum()

total_companies_revenue_value = total_companies_revenue_by_sector['Revenue_num_tradingview'].sum()

total_companies_revenue_value = total_companies_revenue_by_sector['Revenue_num_tradingview'].sum()

total_companies_count_by_sector = merged_df[['Sector1', 'company_id']].groupby('Sector1').count()



## Compute companies count and revenue for net zero companies only

In [5]:

COLS_TO_USE = {
'targets_state':['company_id', 'net0_valid'],
'grouping':['company_id', 'Sector1'],
'companies':['company_id', 'SP100'],
'financials':['company_id', 'Revenue_num_tradingview']
}

all_df = []
for sheetname, cols in COLS_TO_USE.items():
    all_df.append(pd.read_excel(
        XLSX_PATH, 
        sheet_name = sheetname,
        engine = 'openpyxl', 
        usecols = cols
        ))
merged_df = all_df[0]
for i in range(len(COLS_TO_USE) - 1):
    merged_df = pd.merge(
        left = merged_df,
        right = all_df[i+1],
        how="left",
        on="company_id"
    )

cond1 = (merged_df['SP100'] == 1)
cond2 = (merged_df['net0_valid'] == "yes")

filter_cond = cond1 & cond2
filter_cond = filter_cond

merged_df = merged_df.loc[filter_cond]
net_zero_count_by_sector = merged_df[['company_id','Sector1']].groupby('Sector1').count()
net_zero_count = len(merged_df)
net_zero_revenue = merged_df['Revenue_num_tradingview'].sum()
net_zero_revenue_by_sector = merged_df[['Revenue_num_tradingview', 'Sector1']].groupby('Sector1').sum()


#a =  merged_df[merged_df['company_id']==company_id]['Sector1']

## Computing weights by sector

In [6]:
total_companies_revenue_by_sector

,Revenue_num_tradingview
Sector1,
Communication Services,8.785460e+11
Consumer Discretionary,9.829460e+11
Consumer Staples,1.038666e+12
Energy,3.848980e+11
Financial Services,1.019178e+12
Healthcare,8.948460e+11
Industrials,6.091650e+11
Materials,8.618200e+10
Real Estate,1.255000e+10


In [7]:
net_zero_revenue_by_sector

,Revenue_num_tradingview
Sector1,
Communication Services,7.851530e+11
Consumer Discretionary,6.424890e+11
Consumer Staples,7.982990e+11
Energy,9.356100e+10
Financial Services,6.655550e+11
Healthcare,2.914750e+11
Industrials,2.912110e+11
Materials,3.854200e+10
Technology,5.601850e+11


In [8]:
net_zero_revenue_weight_by_sector = net_zero_revenue_by_sector / total_companies_revenue_by_sector
net_zero_count_weight_by_sector = net_zero_count_by_sector / total_companies_count_by_sector

In [9]:
net_zero_count_weight_by_sector

,company_id
Sector1,
Communication Services,0.777778
Consumer Discretionary,0.400000
Consumer Staples,0.636364
Energy,0.250000
Financial Services,0.705882
Healthcare,0.533333
Industrials,0.416667
Materials,0.333333
Real Estate,NaN


In [10]:
net_zero_count_median = net_zero_count / total_companies_count * 100
net_zero_revenue_median = net_zero_revenue / total_companies_revenue_value * 100

In [11]:
net_zero_revenue_median

59.089793470656375

In [12]:
print (list(net_zero_count_weight_by_sector['company_id']))

[0.7777777777777778, 0.4, 0.6363636363636364, 0.25, 0.7058823529411765, 0.5333333333333333, 0.4166666666666667, 0.3333333333333333, nan, 0.3076923076923077, 0.25]


In [13]:
import plotly.graph_objects as go

sectors = net_zero_count_weight_by_sector.index
net_zero_count = net_zero_count_weight_by_sector['company_id'] * 100
net_zero_revenue = net_zero_revenue_weight_by_sector['Revenue_num_tradingview'] * 100

fig = go.Figure()

fig.add_vline(x=net_zero_count_median, y0 = sectors[0], y1 = sectors[-1], line_width=3, line_dash="dash", line_color="#9fb100")
fig.add_vline(x=net_zero_revenue_median, y0 = sectors[0], y1 = sectors[-1], line_width=3, line_dash="dash", line_color="#2000b1")

fig.add_trace(go.Scatter(
    x=net_zero_count,
    y=sectors,
    name='Percent of net zero companies',
    marker=dict(
        color='#9fb100',
        line_color='rgba(156, 165, 196, 1.0)',
    )
))
fig.add_trace(go.Scatter(
    x=net_zero_revenue, y=sectors,
    name='Percent of revenue of net zero companies',
    marker=dict(
        color='#2000b1',
        line_color='rgba(217, 217, 217, 1.0)'
    )
))


fig.update_traces(mode='markers', marker=dict(line_width=1, symbol='circle', size=16))

fig.update_layout(
    title="Percentage of Net Zero Companies Across Sectors",
    xaxis=dict(
        showgrid=False,
        showline=True,
        linecolor='rgb(102, 102, 102)',
        tickfont_color='rgb(102, 102, 102)',
        showticklabels=True,
        dtick=10,
        ticks='outside',
        tickcolor='rgb(102, 102, 102)',
    ),
    margin=dict(l=140, r=40, b=50, t=80),
    legend=dict(
        font_size=10,
        yanchor='middle',
        xanchor='right',
    ),
    width=800,
    height=600,
    paper_bgcolor='white',
    plot_bgcolor='white',
    hovermode='closest',
)


fig.show()

In [14]:
config = {'displaylogo': False,
        'responsive': True
        }

name_fig = 'net_zero_pct_by_sector'


fig.write_image(
                "../../../django_project/static/django_project/images/charts/{}/{}.jpeg".format(
                    'general',
                    name_fig),
                scale=1,
                )
plot(
    fig,
    config=config,
    filename = '../../../django_project/templates/django_project/home/charts/html_exports/{}/{}.html'.format(
        'general',
        name_fig),
    auto_open=False)

'../../../django_project/templates/django_project/home/charts/html_exports/general/net_zero_pct_by_sector.html'

In [15]:
# table for top 10 Revenue without net zero emissions targets + ranking

In [43]:
COLS_TO_USE = {
'companies':['company_id','company_name', 'SP100'],
'targets_state':['company_id', 'net0_valid'],
'financials':['company_id', 'Revenue_num_tradingview'],
'corp_scores':['company_id', 'rank'],
}

all_df = []
for sheetname, cols in COLS_TO_USE.items():
    all_df.append(pd.read_excel(
        XLSX_PATH, 
        sheet_name = sheetname,
        engine = 'openpyxl', 
        usecols = cols
        ))
merged_df = all_df[0]
for i in range(len(COLS_TO_USE) - 1):
    merged_df = pd.merge(
        left = merged_df,
        right = all_df[i+1],
        how="left",
        on="company_id"
    )
# Select Final Figures and Year 2019

cond1 = (merged_df['SP100'] == 1)
cond2 = (merged_df['net0_valid'] != "yes")

filter_cond = cond1 & cond2
filter_cond = filter_cond

merged_df = merged_df.loc[filter_cond]
#net_zero_count_by_sector = merged_df[['company_id','Sector1']].groupby('Sector1').count()
#net_zero_count = len(merged_df)
#net_zero_revenue = merged_df['Revenue_num_tradingview'].sum()
#net_zero_revenue_by_sector = merged_df[['Revenue_num_tradingview', 'Sector1']].groupby('Sector1').sum()


#a =  merged_df[merged_df['company_id']==company_id]['Sector1']

In [47]:
top10_df = merged_df.sort_values('Revenue_num_tradingview', ascending=False).iloc[:10,:][['company_id','company_name','Revenue_num_tradingview','rank']]

In [68]:
top10_df['company_id'] = pd.to_numeric(top10_df['company_id'], downcast='integer')
top10_df['rank'] = pd.to_numeric(top10_df['rank'], downcast='integer')

In [69]:
top10_df

,company_id,company_name,Revenue_num_tradingview,rank
33,34,CVS Health,2.686540e+11,21
87,88,Texas Instruments,2.571410e+11,83
16,17,Berkshire Hathaway Inc.,2.455100e+11,96
41,42,Exxon Mobil,1.781700e+11,90
32,33,Costco,1.667610e+11,92
97,98,Walgreens Boots Alliance,1.395370e+11,76
50,51,Home Depot,1.321100e+11,49
25,26,Chevron,9.440200e+10,90
59,60,"Lowe's Companies, Inc.",8.959700e+10,54
46,47,General Electric,7.989300e+10,92


In [70]:
name_json = 'top10_wo_net_zero'
filename = "../../../django_project/static/django_project/data/{}.json".format(
    name_json)
json = top10_df.to_json(path_or_buf = filename, orient = 'records')

In [71]:
json

In [62]:
import json
 
# Opening JSON file
f = open('essai.json',)
 
# returns JSON object as
# a dictionary
data = json.load(f)
 
# Iterating through the json
# list
for i in data:
    print(i['company_id'])
 
# Closing file
f.close()

34
88
17
42
33
98
51
26
60
47
